In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Plant_village/plant_village.zip -d /content/

In [ ]:
import pandas as pd
import numpy as np
import os
import PIL
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
path='/content/PlantVillage'

In [ ]:
# training data
img_height,img_width=180,180
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  path ,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 20638 files belonging to 15 classes.
Using 16511 files for training.


In [ ]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
# testing data(validation data)
img_height,img_width=180,180
batch_size=32
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  path ,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 20638 files belonging to 15 classes.
Using 4127 files for validation.


In [ ]:
# displaying categories
cn=train_ds.class_names
print(cn)

['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [ ]:
len(cn)

15

In [ ]:
def alexnet_model(input_shape=(180, 180, 3), num_classes=15):
  model = Sequential()
 # model.add(experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3))),
  # we have not return input shape
  model.add(Conv2D(96, (11,11),strides=(4,4) , padding='same', activation='relu',input_shape=input_shape)),
  model.add(MaxPooling2D((3,3),strides=(2,2))),

  model.add(Conv2D(256, (5,5), padding='same', activation='relu')),
  model.add(MaxPooling2D((3,3),strides=(2,2))),

  # multiple convulational layer
  model.add(Conv2D(384, (3,3), padding='same', activation='relu')),
  model.add(Conv2D(384, (3,3), padding='same', activation='relu')),
  model.add(Conv2D(384, (3,3), padding='same', activation='relu')),
  model.add(MaxPooling2D((3,3),strides=(2,2))),
  # adding fully connected layer

  model.add(Flatten()),
  model.add(Dense(4096, activation='relu')),
  model.add(Dropout(0.5)),
  model.add(Dense(4096, activation='relu')),
  model.add(Dropout(0.5)),
  model.add(Dense(num_classes,activation='softmax'))

  return model
  model.summary()

In [ ]:
#model compiling
model = alexnet_model(input_shape=(180, 180, 3), num_classes=15)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#training the model
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


516/516 [==============================] - 35s 54ms/step - loss: 3.3965 - accuracy: 0.1500 - val_loss: 2.5962 - val_accuracy: 0.1003
Epoch 2/10
516/516 [==============================] - 27s 52ms/step - loss: 2.5670 - accuracy: 0.1556 - val_loss: 2.5576 - val_accuracy: 0.1677
Epoch 3/10
516/516 [==============================] - 25s 48ms/step - loss: 2.6012 - accuracy: 0.1561 - val_loss: 2.5660 - val_accuracy: 0.1648
Epoch 4/10
516/516 [==============================] - 25s 49ms/step - loss: 2.5704 - accuracy: 0.1531 - val_loss: 2.5639 - val_accuracy: 0.1648
Epoch 5/10
516/516 [==============================] - 25s 49ms/step - loss: 2.5693 - accuracy: 0.1531 - val_loss: 2.5636 - val_accuracy: 0.1648
Epoch 6/10
516/516 [==============================] - 25s 48ms/step - loss: 2.5689 - accuracy: 0.1531 - val_loss: 2.5635 - val_accuracy: 0.1648
Epoch 7/10
516/516 [==============================] - 25s 48ms/step - loss: 2.5691 - accuracy: 0.1531 - val_loss: 2.5632 - val_accuracy: 0.1648
Epo

In [ ]:
def predict_image(img):
  img_4d=img.reshape(-1,180,180,3)
  prediction=model.predict(img_4d)[0]
  return {class_names[i]: float(prediction[i]) for i in range(15)}

In [ ]:
import gradio as gr

In [ ]:
image = gr.Image()
label = gr.Label(num_top_classes=15)

gr.Interface(fn=predict_image, inputs=image, outputs=label).launch(share='True')